<a href="https://colab.research.google.com/github/alihammad/Python_snippets/blob/master/BigQuery_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps

1.   Mount the Google Drive
2.   Download the Google credentials Json file
3.   Upload the credentials file
4.   Set up the environment for authentication and authorization 
5.   Create BigQuery client object to access Google open data set(Hacker News)
6.   Read the schema, tables and tables data




## Mount the Google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**Go to this page and read the section *Creating a service account* to download your credentials as a json file.** 



## Upload the credentials file


In [0]:
from google.colab import files
files.upload()

## Setup the  environment

In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "test-82d4554be7a8.json"

## Interact with Google Open dataset (Hacker News)

In [0]:
from google.cloud import bigquery

In [0]:
client = bigquery.Client()

In [0]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [14]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

comments
full
full_201510
stories


In [0]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

In [16]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ()),
 SchemaField(

In [17]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,Selenoid: lightning fast Docker-based Selenium...,https://github.com/aerokube/selenoid,None,None,vania-pooh,2.0,1531827823,2018-07-17 11:43:43+00:00,story,17548966,NaN,0.0,None,None
1,None,None,You're right. I was looking at the diagram. Cl...,None,brucehart,NaN,1345732908,2012-08-23 14:41:48+00:00,comment,4422609,4422196.0,NaN,None,None
2,None,None,Nothing but a small set of scattered anecdotal...,None,anongraddebt,NaN,1544737162,2018-12-13 21:39:22+00:00,comment,18676377,18671189.0,NaN,None,None
3,None,None,&gt; Is fb actively stifling competition? Is t...,None,whyenot,NaN,1552357349,2019-03-12 02:22:29+00:00,comment,19364804,19364278.0,NaN,None,None
4,None,None,"I think most of those are past the ""Risk your ...",None,dkhenry,NaN,1357930699,2013-01-11 18:58:19+00:00,comment,5044306,5044293.0,NaN,None,None


In [18]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,title
0,Selenoid: lightning fast Docker-based Selenium...
1,None
2,None
3,None
4,None
